In [ ]:
import polars as pl
from pathlib import Path
import pandas as pd
import collections
import pyarrow as pa
import pyarrow.parquet as pq
from typing import List, Dict
from vkai.connecting import SparkIgnitor

In [ ]:
spark = SparkIgnitor()

In [ ]:
# client = spark.connect()

In [ ]:
def generate_query_for_device(device:str, limit=2_500_000):
    m = dict(google="ods_targetdb_data.google_app", apple="ods_targetdb.apple_app")
    assert device in m.keys(), f"{device} can be either <apple> or <google>"
    table = m[device]
    query = f"""
        SELECT p.title, p.category_id FROM {table} as p
        LIMIT {str(limit)}
    """
    return query

In [ ]:
def request_for_device(client, device:str, limit):
    query = generate_query_for_device(device=device, limit=int(limit))
    handle = client.sql(query)
    response = handle.toPandas()
    _pl_response = pl.from_pandas(response).with_columns([
            pl.lit(device).alias("source")
        ])
    return _pl_response

In [ ]:
# db_ios = request_for_device(client=client, device="apple", limit=1_000_000)
# db_droid = request_for_device(client=client, device="google", limit=1_000_000)
# db = pl.concat([db_ios, db_droid])

In [ ]:
where = Path.home() / "IDataset" / "mobapp" / "mobapp.parquet"
where_ios = Path.home() / "IDataset" / "mobapp" / "ios_iab_category.csv"
where_droid = Path.home() / "IDataset" / "mobapp" / "droid_iab_category.csv"
where_iab = Path.home() / "IDataset" / "mobapp" / "iab_category.csv"

In [ ]:
dp = pq.read_table(where)

In [ ]:
df = pl.from_arrow(dp)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df_ios = pl.read_csv(where_ios)
df_droid = pl.read_csv(where_droid)
df_meta_iab = pl.read_csv(where_iab)

In [ ]:
df_meta_iab.head()

In [ ]:
df_ios.head()
# `category_id` соединяется с основной таблицей (выгрузкой приложух)
# Но если напрямую это сделать, то не будет информации про класс `iab_category_name`. 
# Для этого надо сделать mapping с табличкой `iab_category`. по колонке `iab_category_id`

In [ ]:
df_meta_iab.head()

In [ ]:
print(df_ios.shape)
df_ios.head()

In [ ]:
from vkai.processing import _pl_unique

In [ ]:
df_ios = _pl_unique(df_ios, column_name="category_id")
df_droid = _pl_unique(df_droid, column_name="category_id")

In [ ]:
df_meta_iab_ios = df_meta_iab.rename({"id": "iab_category_id"}).select(["iab_category_id", "tier1", "tier2", "tier3", "parent_id", "name", "description"]).join(df_ios, on="iab_category_id").select(["iab_category_id", "category_id", "parent_id", "name", "description"]).rename({"name": "iab_category_name"})
df_meta_iab_droid = df_meta_iab.rename({"id": "iab_category_id"}).select(["iab_category_id", "tier1", "tier2", "tier3", "parent_id", "name", "description"]).join(df_droid, on="iab_category_id").select(["iab_category_id", "category_id", "parent_id", "name", "description"]).rename({"name": "iab_category_name"})

In [ ]:
db_cat = set(df["category_id"]) # 108
db_meta_iab_ios = set(df_meta_iab_ios["category_id"]) # 58

In [ ]:
db_meta_iab_ios - db_cat

In [ ]:
df_meta_iab_ios.filter(pl.col("iab_category_id") == 239) 
# На одну категорию таксономийную IAB категорию - хобби и интересы
# приходится 8 разных категорий ios (разметка стора).

In [ ]:
from vkai.processing import _pl_count

In [ ]:
_pl_count(df_meta_iab_ios, column_name="iab_category_id")

In [ ]:
df_droid.head()

In [ ]:
df_droid.sort("category_id")

In [ ]:
df_meta_iab_droid.sort("category_id")

In [ ]:
def build_graph(_pl_graph_meta):
    import networkx as nx
    cix, pix, nix = list(_pl_graph_meta["iab_category_id"]), list(_pl_graph_meta["parent_id"]), list(_pl_graph_meta["description"])
    g = nx.DiGraph()
    for ci, pi, ni in zip(cix, pix, nix):
        g.add_node(ci, label=ni)
        if pi != 0:
            g.add_edge(pi, ci)
    return g

In [ ]:
_g_droid = build_graph(df_meta_iab_droid)

In [ ]:
import networkx as nx

In [ ]:
# 1. type -> application vs album vs game
# 2. category

In [ ]:
from vkai.processing import _pl_project, _pl_count

In [ ]:
_pl_count

In [ ]:
response_ios = _pl_count(df.join(df_meta_iab_ios, on="category_id"), column_name="iab_category_name")

In [ ]:
response_droid = _pl_count(df.join(df_meta_iab_droid, on="category_id"), column_name="iab_category_name")

In [ ]:
def _pl_indicator(db, proj, on:str, probs_col:str = "description"):
    projected = db.join(proj, on=on).with_row_count().with_columns([
        pl.count("row_nr").over(probs_col).alias("counts_per_iab"),
        pl.first("row_nr").over(probs_col).alias("mask")
    ]).filter(pl.col("mask") == pl.col("row_nr")).sort("counts_per_iab", descending=True)
    
    return projected
    # df.rename({"category_id": "iab_category_id"}).join(df_meta_iab_ios, on="iab_category_id")

In [ ]:
# db_droid = _pl_indicator(df, df_meta_iab_droid, on="category_id", probs_col="iab_category_name", title="Android")

In [ ]:
# db_ios = _pl_indicator(df, df_meta_iab_ios, on="category_id", probs_col="iab_category_name", title="iOS")

In [ ]:
# probs_ios = db_ios.with_row_count().with_columns([
#     pl.count("row_nr").over("description_right").alias("counts"),
#     pl.first("row_nr").over("description_right").alias("mask_counts")
# ]).filter(pl.col("mask_counts") == pl.col("row_nr"))

In [ ]:
# probs_droid = db_droid.with_row_count().with_columns([
#     pl.count("row_nr").over("description_right").alias("counts"),
#     pl.first("row_nr").over("description_right").alias("mask_counts")
# ]).filter(pl.col("mask_counts") == pl.col("row_nr"))

In [ ]:
probs_ios_x, probs_ios_y = list(response_ios["iab_category_name"]), list(response_ios["counts_per_iab_category_name"])
probs_droid_x, probs_droid_y = list(response_droid["iab_category_name"]), list(response_droid["counts_per_iab_category_name"])

In [ ]:
def jotai(xs, ys, title="Y"):
    import plotly.express as px
    fig = px.bar(x=xs, y=ys)
    fig.update_layout(
        title=dict(text=title, font=dict(size=14), automargin=False, yref='paper'),
            xaxis={'type': 'category'}
        )
    fig.update_layout(yaxis_title=None)
    fig.update_layout(xaxis_title=None)
    fig.update_xaxes(tickfont_size=9, ticks="outside", ticklen=0.5, tickwidth=1)
    return fig

In [ ]:
fig_ios = jotai(probs_ios_x, probs_ios_y, "IOS по разметке IAB")
fig_droid = jotai(probs_droid_x, probs_droid_y, "ANDROID по разметке IAB")

In [ ]:
fig_ios

In [ ]:
fig_droid

In [ ]:
import plotly.io as pio

In [ ]:
pio.write_image(fig_ios, file="ios_iab_name.png", scale=5, engine="kaleido")
pio.write_image(fig_droid, file="droid_iab_name.png", scale=5, engine="kaleido")

In [ ]:
len(set(probs_droid_x))

In [ ]:
set(probs_ios_x).difference(probs_droid_x)

In [ ]:
list(
    df.with_columns([
    pl.col("title").str.contains("Недвижимость").alias("_mask1"),
    pl.col("source").str.contains("apple").alias("_mask2")
]).filter(pl.col("_mask1") & pl.col("_mask2")).join(df_meta_iab_ios, on="category_id").select(["iab_category_name"])["iab_category_name"]
)

In [ ]:
list(df.join(df_meta_iab_ios, on="category_id").select(["title", "iab_category_name"]).with_columns([
    pl.col("iab_category_name").str.contains("Movies").alias("mask")
]).filter("mask")["title"])

In [ ]:
_df = _pl_count(df, column_name="category_id")

In [ ]:
_df

In [ ]:
_df.select(["category_id", "counts_per_category_id"]).head()

In [ ]:
import plotly.express as px
import plotly.io as pio

In [ ]:
_df.filter(pl.col("category") == "Спорт")

In [ ]:
_df = _df.sort("category_id")

In [ ]:
px.line(x=_df["category_id"], y=_df["counts_per_category_id"], color_discrete_sequence=px.colors.sequential.GnBu_r)